### 1. Setup & Imports

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from src.data_collection.market_data import MarketDataCollector
from src.data_collection.news_scraper import NewsScraper
from src.sentiment.finbert_engine import FinBERTSentimentEngine
from src.utils.preprocessing import DataPreprocessor
from src.models.dual_lstm import DualLSTMModel
from src.agents.kelly_agent import KellyCriterionAgent

# Display settings
pd.set_option('display.max_columns', None)
sns.set_style('whitegrid')

print("✅ Imports successful!")

### 2. Collect Market Data

In [ ]:
# Initialize collector
collector = MarketDataCollector()

# Collect data for one stock (faster for testing)
reliance_data = collector.download_stock_data('RELIANCE.NS')
reliance_data = collector.add_technical_indicators(reliance_data)
reliance_data = collector.create_target_variable(reliance_data)
reliance_data = reliance_data.dropna()

print(f"\n📊 Collected {len(reliance_data)} days of data")
reliance_data.tail()

### 3. Visualize Technical Indicators

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(15, 10))

# Price + Bollinger Bands
axes[0].plot(reliance_data.index, reliance_data['Close'], label='Close', linewidth=2)
axes[0].plot(reliance_data.index, reliance_data['BBU_20_2.0'], '--', label='BB Upper', alpha=0.7)
axes[0].plot(reliance_data.index, reliance_data['BBL_20_2.0'], '--', label='BB Lower', alpha=0.7)
axes[0].set_title('Price with Bollinger Bands')
axes[0].legend()
axes[0].grid(True)

# RSI
axes[1].plot(reliance_data.index, reliance_data['RSI_14'], color='purple', linewidth=2)
axes[1].axhline(70, color='r', linestyle='--', alpha=0.5)
axes[1].axhline(30, color='g', linestyle='--', alpha=0.5)
axes[1].set_title('RSI (14)')
axes[1].set_ylim(0, 100)
axes[1].grid(True)

# MACD
axes[2].plot(reliance_data.index, reliance_data['MACD_12_26_9'], label='MACD', linewidth=2)
axes[2].plot(reliance_data.index, reliance_data['MACDs_12_26_9'], label='Signal', linewidth=2)
axes[2].axhline(0, color='k', linestyle='-', alpha=0.3)
axes[2].set_title('MACD')
axes[2].legend()
axes[2].grid(True)

plt.tight_layout()
plt.show()

### 4. Collect & Analyze News Sentiment

In [ ]:
# Scrape news
scraper = NewsScraper()
news_df = scraper.fetch_all_news()

print("\n📰 Sample Headlines:")
news_df[['date_only', 'title']].head(10)

In [ ]:
# Analyze sentiment (this will download FinBERT-India on first run)
engine = FinBERTSentimentEngine()
news_df = engine.analyze_dataframe(news_df)

# Aggregate daily
daily_sentiment = engine.aggregate_daily_sentiment(news_df)

daily_sentiment.head()

In [ ]:
# Plot sentiment trend
plt.figure(figsize=(15, 5))
plt.plot(daily_sentiment['date'], daily_sentiment['sentiment_mean'], 
         marker='o', label='Daily Sentiment', alpha=0.6)
plt.plot(daily_sentiment['date'], daily_sentiment['sentiment_ma_5'], 
         linewidth=2, label='5-Day MA', color='orange')
plt.axhline(0, color='gray', linestyle='--', alpha=0.5)
plt.title('News Sentiment Over Time')
plt.xlabel('Date')
plt.ylabel('Sentiment Score')
plt.legend()
plt.grid(True)
plt.show()

### 5. Test Kelly Criterion Agent

In [ ]:
agent = KellyCriterionAgent()

# Test different scenarios
capital = 100000

print("\n" + "="*60)
print("Testing Kelly Criterion Agent")
print("="*60)

# Scenario 1: High confidence + aligned sentiment
rec1 = agent.get_recommendation(0.75, 0.6, capital, "RELIANCE", verbose=True)

# Scenario 2: High confidence + conflicting sentiment
rec2 = agent.get_recommendation(0.70, -0.4, capital, "TCS", verbose=True)

# Scenario 3: Low confidence
rec3 = agent.get_recommendation(0.55, 0.2, capital, "INFY", verbose=True)

### 6. Model Architecture Overview

In [ ]:
# Build model to see architecture
model = DualLSTMModel()

price_shape = (30, 10)  # 30 days, 10 features
sentiment_shape = (2,)   # 2 sentiment features

model.build_model(price_shape, sentiment_shape)

### 7. Data Preprocessing Pipeline

In [ ]:
# For full training, use the train.py script
# This is just to demonstrate the preprocessing

preprocessor = DataPreprocessor()

# Create dummy market data for demo
market_df_demo = reliance_data.copy()

# Merge with sentiment
merged = preprocessor.merge_market_and_sentiment(market_df_demo, daily_sentiment)

print("\n📊 Merged Data Shape:", merged.shape)
merged[['Close', 'sentiment_mean', 'Target']].tail(10)

### 8. Feature Importance Analysis

In [ ]:
# Correlation heatmap
features = ['Close', 'Volume', 'RSI_14', 'MACD_12_26_9', 'ATRr_14', 'sentiment_mean', 'Target']
available_features = [f for f in features if f in merged.columns]

plt.figure(figsize=(10, 8))
corr = merged[available_features].corr()
sns.heatmap(corr, annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Feature Correlation Heatmap')
plt.tight_layout()
plt.show()

### 9. Next Steps

For full training and deployment:

```bash
# Collect all stocks
python src/data_collection/market_data.py
python src/data_collection/news_scraper.py
python src/sentiment/finbert_engine.py

# Train model
python src/train.py

# Make predictions
python src/predict.py

# Launch dashboard
streamlit run dashboard.py
```